In [1]:
import pandas as pd
import polars as pl
df = pd.read_parquet("/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet")
df_pl = pl.read_parquet("/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet")

In [2]:
# %%writefile info_checker.py
# # script.py

# import argparse
# import polars as pl

# def load_large_parquet(file_path):
#     """
#     Load a large Parquet DataFrame using Polars.

#     Args:
#         file_path (str): Path to the Parquet file.

#     Returns:
#         pl.DataFrame: Loaded DataFrame.
#     """
#     try:
#         # Use Polars for fast Parquet file loading
#         df = pl.read_parquet(file_path)
#         return df
#     except Exception as e:
#         print(f"Error loading Parquet file: {e}")
#         return None

# def find_nan_summary(df):
#     """
#     Find columns with NaN values and count the number of NaN rows for each.

#     Args:
#         df (pl.DataFrame): DataFrame to analyze.

#     Returns:
#         dict: A dictionary with column names as keys and NaN row counts as values.
#     """
#     try:
#         nan_summary = {col: df[col].null_count() for col in df.columns if df[col].null_count() > 0}
        
#         return nan_summary
#     except Exception as e:
#         print(f"Error computing NaN summary: {e}")
#         return {}

# def print_nan_summary(nan_summary):
#     """
#     Print the columns with NaN values and their respective counts.

#     Args:
#         nan_summary (dict): Dictionary with column names and their NaN row counts.
#     """
#     if nan_summary:
        
#         print("Columns with NaN values and their NaN row counts:")
        
#         for col, count in nan_summary.items():
#             print(f"- {col}: {count} NaN rows - {} Correct {}")
#     else:
#         print("No columns with NaN values found.")

# def main():
#     # Parse arguments
#     parser = argparse.ArgumentParser(description="Load a large Parquet file and analyze NaN values.")
#     parser.add_argument("--data_path", required=True, help="Path to the Parquet file.")
#     args = parser.parse_args()

#     # Load DataFrame
#     df = load_large_parquet(args.data_path)

#     if df is not None:
#         # Find NaN summary and print it
#         nan_summary = find_nan_summary(df)
#         print_nan_summary(nan_summary)
#     else:
#         print("DataFrame could not be loaded.")

# if __name__ == "__main__":
#     main()

In [3]:
# !python info_checker.py --data_path "/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet"

In [4]:
df.head()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,0,0,1,3.889038,NaN,NaN,NaN,NaN,NaN,0.851033,...,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504
1,0,0,7,1.370613,NaN,NaN,NaN,NaN,NaN,0.676961,...,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639
2,0,0,9,2.285698,NaN,NaN,NaN,NaN,NaN,1.056285,...,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828
3,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516
4,0,0,14,0.440570,NaN,NaN,NaN,NaN,NaN,0.955200,...,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000


In [5]:
df.columns

Index(['date_id', 'time_id', 'symbol_id', 'weight', 'feature_00', 'feature_01',
       'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06',
       'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11',
       'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16',
       'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26',
       'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31',
       'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36',
       'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41',
       'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46',
       'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51',
       'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56',
       'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_6

- `date_id` and `time_id` - Integer values that are ordinally sorted, providing a chronological structure to the data, although the **actual time intervals** between `time_id` values may vary.
- `symbol_id` - Identifies a unique financial instrument. Each row in the dataset corresponds to a unique combination of a symbol (identified by `symbol_id`) and a timestamp (represented by `date_id` and `time_id`). The `symbol_id` column contains encrypted identifiers. Each `symbol_id` is not guaranteed to appear in all `time_id` and `date_id` combinations.
- `weight` - The weighting used for calculating the scoring function.
- `feature_{00...78}` - Anonymized market data.
- `responder_{0...8}` - Anonymized responders clipped between -5 and 5. The responder_6 field is what you are trying to predict.

In [6]:
TEMPORAL_FEATURES = ['date_id', 'time_id','symbol_id']
MARKET_FEATURES = [f'feature_{i:02}' for i in range(0,79)]
RESPONDER_FEATURES = [f'responder_{i}' for i in range(0,9)]
RESPONDER_FEATURES.remove('responder_6')
SYMBOL_FEATURES = ['symbol_id']
WEIGHT = ['WEIGHT']

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1944210 entries, 0 to 1944209
Data columns (total 92 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date_id      int16  
 1   time_id      int16  
 2   symbol_id    int8   
 3   weight       float32
 4   feature_00   float32
 5   feature_01   float32
 6   feature_02   float32
 7   feature_03   float32
 8   feature_04   float32
 9   feature_05   float32
 10  feature_06   float32
 11  feature_07   float32
 12  feature_08   float32
 13  feature_09   int8   
 14  feature_10   int8   
 15  feature_11   int16  
 16  feature_12   float32
 17  feature_13   float32
 18  feature_14   float32
 19  feature_15   float32
 20  feature_16   float32
 21  feature_17   float32
 22  feature_18   float32
 23  feature_19   float32
 24  feature_20   float32
 25  feature_21   float32
 26  feature_22   float32
 27  feature_23   float32
 28  feature_24   float32
 29  feature_25   float32
 30  feature_26   float32
 31  feature_27   float32
 32

In [8]:
df.describe()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
count,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,0.0,0.0,0.0,0.0,0.0,1.944210e+06,...,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06
mean,9.384629e+01,4.240000e+02,1.376638e+01,1.973281e+00,NaN,NaN,NaN,NaN,NaN,-4.463175e-02,...,-9.805073e-02,8.424639e-03,1.076465e-02,2.412764e-03,1.139089e-02,2.185480e-02,3.326982e-03,1.487634e-03,-4.817980e-04,1.078154e-03
std,4.813196e+01,2.450851e+02,1.108778e+01,9.691969e-01,NaN,NaN,NaN,NaN,NaN,9.479579e-01,...,6.398993e-01,9.558780e-01,1.141793e+00,8.442027e-01,1.276044e+00,1.266448e+00,1.225165e+00,8.705768e-01,8.918150e-01,8.737320e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,4.405696e-01,NaN,NaN,NaN,NaN,NaN,-1.176608e+01,...,-3.393299e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00
25%,5.400000e+01,2.120000e+02,7.000000e+00,1.323803e+00,NaN,NaN,NaN,NaN,NaN,-4.756072e-01,...,-3.195696e-01,-2.322211e-01,-2.667868e-01,-1.210319e-01,-4.444537e-01,-5.146699e-01,-2.569897e-01,-3.558709e-01,-3.914038e-01,-2.892584e-01
50%,9.900000e+01,4.240000e+02,1.200000e+01,1.763827e+00,NaN,NaN,NaN,NaN,NaN,-5.818180e-02,...,-2.468792e-01,-3.943805e-03,-2.333469e-02,-1.248489e-03,-1.031224e-02,-1.836913e-02,-4.690550e-03,-9.597129e-03,-2.376243e-02,-3.107830e-04
75%,1.360000e+02,6.360000e+02,1.700000e+01,2.393846e+00,NaN,NaN,NaN,NaN,NaN,3.493771e-01,...,-1.256560e-01,2.309678e-01,2.538152e-01,1.191232e-01,4.292141e-01,5.065366e-01,2.438874e-01,3.360999e-01,3.463979e-01,2.840819e-01
max,1.690000e+02,8.480000e+02,3.800000e+01,6.011999e+00,NaN,NaN,NaN,NaN,NaN,1.532000e+01,...,4.370195e+01,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


In [9]:
empty_columns = []
fully_filled_columns = []
partially_empty_columns = []

In [10]:
## pl
for feature in df_pl.columns:
    # Count empty and non-empty rows
    empty_rows = df_pl[feature].is_null().sum()
    nonempty_rows = len(df_pl[feature]) - empty_rows

    # Classify the columns based on the counts
    if nonempty_rows == 0:
        empty_columns.append(feature)
    elif empty_rows == 0:
        fully_filled_columns.append(feature)
    else:
        partially_empty_columns.append(feature)

    # Print feature statistics
    print(f'{feature} : total - {len(df_pl[feature])} - empty - {empty_rows} - nonempty - {nonempty_rows}')


date_id : total - 1944210 - empty - 0 - nonempty - 1944210
time_id : total - 1944210 - empty - 0 - nonempty - 1944210
symbol_id : total - 1944210 - empty - 0 - nonempty - 1944210
weight : total - 1944210 - empty - 0 - nonempty - 1944210
feature_00 : total - 1944210 - empty - 1944210 - nonempty - 0
feature_01 : total - 1944210 - empty - 1944210 - nonempty - 0
feature_02 : total - 1944210 - empty - 1944210 - nonempty - 0
feature_03 : total - 1944210 - empty - 1944210 - nonempty - 0
feature_04 : total - 1944210 - empty - 1944210 - nonempty - 0
feature_05 : total - 1944210 - empty - 0 - nonempty - 1944210
feature_06 : total - 1944210 - empty - 0 - nonempty - 1944210
feature_07 : total - 1944210 - empty - 0 - nonempty - 1944210
feature_08 : total - 1944210 - empty - 16980 - nonempty - 1927230
feature_09 : total - 1944210 - empty - 0 - nonempty - 1944210
feature_10 : total - 1944210 - empty - 0 - nonempty - 1944210
feature_11 : total - 1944210 - empty - 0 - nonempty - 1944210
feature_12 : to

In [11]:
# for feature in list(df.columns):
#     empty_rows = sum(df[[feature]].isna().values)[0]
#     nonempty_rows = len(df[[feature]]) - empty_rows

#     ## store in feature_stats since recalculating empty rows for all 79 features takes a while
#     if nonempty_rows == 0:
#         empty_columns.append(feature)

#     elif empty_rows == 0:
#         fully_filled_columns.append(feature)

#     else:
#         partially_empty_columns.append(feature)
    
#     print(f'{feature} : total  - {len(df[[feature]])} - empty - {empty_rows} - nonempty - {nonempty_rows}')

So we have 3 cases
1. Fully Empty columns : we can just drop them
2. Partially Empty Columns: we can impute them
3. Full Columns : No imputation necessary

In [12]:
df.sort_values(['time_id','date_id'])

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,0,0,1,3.889038,NaN,NaN,NaN,NaN,NaN,0.851033,...,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504
1,0,0,7,1.370613,NaN,NaN,NaN,NaN,NaN,0.676961,...,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639
2,0,0,9,2.285698,NaN,NaN,NaN,NaN,NaN,1.056285,...,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828
3,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516
4,0,0,14,0.440570,NaN,NaN,NaN,NaN,NaN,0.955200,...,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944205,169,848,19,3.438631,NaN,NaN,NaN,NaN,NaN,-0.028087,...,-0.166964,0.983339,-0.669860,0.272615,-3.676842,-1.221126,1.070584,0.465345,0.207483,0.874975
1944206,169,848,30,0.768528,NaN,NaN,NaN,NaN,NaN,-0.022584,...,-0.352810,0.992615,0.961595,1.089402,0.796034,0.488380,1.846634,-0.088542,-0.008324,-0.153451
1944207,169,848,33,1.354696,NaN,NaN,NaN,NaN,NaN,-0.024804,...,-0.239716,1.701618,0.757672,-5.000000,-3.174266,-1.110790,-3.349107,-0.407801,-0.185842,-0.931004
1944208,169,848,34,1.021797,NaN,NaN,NaN,NaN,NaN,-0.016138,...,-0.442859,-2.036891,-0.064228,1.919665,1.827681,0.872019,3.248694,0.254584,0.090288,0.434726


In [13]:
df[df.date_id==0].symbol_id.unique()

array([ 1,  7,  9, 10, 14, 16, 19, 33], dtype=int8)

In [14]:
df[df.date_id==1].symbol_id.unique()

array([ 0,  1,  2,  7,  9, 10, 13, 15, 16, 19, 33, 38], dtype=int8)

In [15]:
df[df.symbol_id == 1].date_id.unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169],
      dtype=int16)

In [16]:
set(df[df.symbol_id == 1].date_id.unique()) - set(df[df.symbol_id == 7].date_id.unique()) 

{53, 71, 97}

In [17]:
mask1 = (df.symbol_id == 1) & (df.date_id == 0)
mask2 = (df.symbol_id == 7) & (df.date_id == 0)
list(set(df[mask1].time_id.unique()) - set(df[mask2].date_id.unique()))[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

## Splitting Temporally

In [18]:
df = df.sort_values(['date_id', 'time_id'])
date_counts = df.date_id.value_counts()

In [19]:
date_counts = pd.DataFrame(date_counts.sort_index())
date_counts['cumulative_sum'] = date_counts['count'].cumsum()
date_counts.head()

,count,cumulative_sum
date_id,,
0,6792,6792
1,10188,16980
2,9339,26319
3,10188,36507
4,10188,46695


We want to do a 80-20 split based on date

In [20]:
total = len(df)
split = 0.8
apprx_train_len = int(total*split)
apprx_test_len = total - apprx_train_len
date_counts['Train'] = date_counts['cumulative_sum'] < apprx_train_len
date_counts.head()

,count,cumulative_sum,Train
date_id,,,
0,6792,6792,True
1,10188,16980,True
2,9339,26319,True
3,10188,36507,True
4,10188,46695,True


In [21]:
last_train_data = date_counts[date_counts.Train == True].tail(1) 
last_train_data

,count,cumulative_sum,Train
date_id,,,
142,16131,1546029,True


In [22]:
first_test_data = date_counts[date_counts.Train == False].head(1)
first_test_data

,count,cumulative_sum,Train
date_id,,,
143,15282,1561311,False


In [23]:
train_len = last_train_data['cumulative_sum'].values[0]
test_len = len(df) - last_train_data['cumulative_sum'].values[0]
print("Training data :", train_len,"-" ,round(train_len*100/len(df),2), "%" )
print("Testing data :", test_len, "-", round(test_len*100/len(df),2), "%")

Training data : 1546029 - 79.52 %
Testing data : 398181 - 20.48 %


In [24]:
df.loc[:, 'Train'] = df.date_id <= last_train_data.index[0]
df.head()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,Train
0,0,0,1,3.889038,NaN,NaN,NaN,NaN,NaN,0.851033,...,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504,True
1,0,0,7,1.370613,NaN,NaN,NaN,NaN,NaN,0.676961,...,2.965889,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639,True
2,0,0,9,2.285698,NaN,NaN,NaN,NaN,NaN,1.056285,...,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828,True
3,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516,True
4,0,0,14,0.440570,NaN,NaN,NaN,NaN,NaN,0.955200,...,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000,True


In [25]:
df = df.drop(empty_columns, axis = 1)
train_df = df[df.Train == True]
test_df = df[df.Train == False]
# train_df.to_csv('train.csv', index = False)
# test_df.to_csv('test.csv', index = False)

## Imputer

Now we want to impute numerical values based on the previous stock info. We check our implementation on a sample example first. So lets take a sample dataset first

In [26]:
# Sample DataFrame
data = {
    'Date': [3, 1, 2, 0, 3, 1, 2, 0],
    'Company': [0, 0, 0, 0, 1, 1, 1, 1],
    'Stock Price': [None, 150.0, None, 152.0, 312.0, None, 310.0, None]
}

data = pd.DataFrame(data)
data.sort_values(['Company','Date'])

,Date,Company,Stock Price
3,0,0,152.0
1,1,0,150.0
2,2,0,NaN
0,3,0,NaN
7,0,1,NaN
5,1,1,NaN
6,2,1,310.0
4,3,1,312.0


So this dataframe has different companies, and some of the values are empty

In [27]:
# Sort by Company and Date to ensure proper ordering
data = data.sort_values(by=['Company', 'Date'])

# Forward fill missing stock prices for each company
data['Stock Price'] = data.groupby('Company')['Stock Price'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

data

/tmp/ipykernel_17/1717159857.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Stock Price'] = data.groupby('Company')['Stock Price'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))


,Date,Company,Stock Price
3,0,0,152.0
1,1,0,150.0
2,2,0,150.0
0,3,0,150.0
7,0,1,310.0
5,1,1,310.0
6,2,1,310.0
4,3,1,312.0


Here we filled the data based on each company's previous stock data. We can apply that to our data too

In [28]:
for feature in partially_empty_columns:
    df[feature] = df.groupby('symbol_id')[feature].transform(lambda x: x.fillna(method='ffill').fillna(method = 'bfill'))

/tmp/ipykernel_17/507490440.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[feature] = df.groupby('symbol_id')[feature].transform(lambda x: x.fillna(method='ffill').fillna(method = 'bfill'))


In [29]:
df.head()

,date_id,time_id,symbol_id,weight,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,...,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,Train
0,0,0,1,3.889038,0.851033,0.242971,0.263400,-0.891687,11,7,...,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504,True
1,0,0,7,1.370613,0.676961,0.151984,0.192465,-0.521729,11,7,...,2.965889,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639,True
2,0,0,9,2.285698,1.056285,0.187227,0.249901,-0.773050,11,7,...,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828,True
3,0,0,10,0.690606,1.139366,0.273328,0.306549,-1.262223,42,5,...,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516,True
4,0,0,14,0.440570,0.955200,0.262404,0.344457,-0.613813,44,3,...,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000,True


In [30]:
# test2 = pd.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet')
# test2.head()

In [31]:
# %load_ext cudf.pandas
# import pandas as pdd